In [112]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
import random,os
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor

In [113]:
train = pd.read_csv("/kaggle/input/widsdataset/train.csv")
test = pd.read_csv("/kaggle/input/widsdataset/test.csv")

In [114]:
train.head()

In [115]:
train.columns

Finding the Missing Values

In [116]:
# Train 
total_train = train.isnull().sum()
percent_train = (train.isnull().sum())*100/(len(train))
na_train = pd.concat([total_train, percent_train], axis=1, keys=['Total', 'Percent'])
na_train = na_train[na_train['Total']>0]
na_train.sort_values('Total', ascending=False, inplace=True)
print(na_train)

In [117]:
# Test
total_test = test.isnull().sum()
percent_test = (test.isnull().sum())*100/(len(train))
na_test = pd.concat([total_test, percent_test], axis=1, keys=['Total', 'Percent'])
na_test = na_test[na_test['Total']>0]
na_test.sort_values('Total', ascending=False, inplace=True)
print(na_train)

Taking Out NA values in top 5 because they are such a large percentage 
also taking out ID because it is not useful for this 

In [118]:
TARGET = "site_eui"
ID = "id"

Making a Correlation Matrix with Matplotlib

In [119]:
plt.style.use('classic')
fig = plt.figure(figsize=(10, 10))
plt.matshow(train.corr(), fignum=fig.number)
plt.colorbar()
plt.show()

This correlation matrix shows that there are a lot of possitively and negatively correlated features. Might need to address multicollinearity in the model. 

There are also a lot of categorical variables. Let's try a catboost model! 

Building the CAT(boost) Model!
![cat](https://media.wired.co.uk/photos/60c8730fa81eb7f50b44037e/3:2/w_3329,h_2219,c_limit/1521-WIRED-Cat.jpeg)

In [122]:
# Making Sure Which Columns Are Which
cat_columns = train.dtypes[train.dtypes == "object"].index.tolist()
num_columns = train.dtypes[train.dtypes != "object"].index.tolist()
print(cat_columns)

In [127]:
SEED = 7495
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
test_size = 0.2

In [128]:
str_list = [] 
num_list = []
for colname, colvalue in train.iteritems():
    if type(colvalue[1]) == str:
        str_list.append(colname)
    else:
        num_list.append(colname)
        
for col in str_list:
    encoder = LabelEncoder()
    encoder.fit(train[col])
    train[col] = encoder.transform(train[col])

    for label in np.unique(test[col]):
        if label not in encoder.classes_: 
            encoder.classes_ = np.append(encoder.classes_, label) 
    test[col] = encoder.transform(test[col])

In [ ]:
X = train.drop([ID, TARGET],axis =1)
y = train[TARGET]

model = CatBoostRegressor(
    verbose=1000,
    early_stopping_rounds=10,
    random_seed=SEED,
    max_depth=12,
    task_type='CPU',
    learning_rate=.025,
    iterations=28000,
    loss_function="RMSE",
    eval_metric= "RMSE"
)
model.fit(X, y)

In [ ]:
X_test = test.drop([ID],axis=1)
pred_test = model.predict(X_test)

In [ ]:
sub = pd.read_csv(/kaggle/input/widsdataset/sample_solution.csv)
sub[TARGET] = pred_test
sub.to_csv(submission_2.csv,index=False)
sub.head()

This is my catboost model! Let's submit and see what happens! :) 
![catboost](https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/best-girl-cat-names-1606245046.jpg?crop=0.668xw:1.00xh;0.126xw,0&resize=640:*)